In [1]:
"""將tensorflow檔轉成tensorflow lite，使機器負載量變小"""
from tensorflow import lite
from tensorflow.keras import models

# Parameters
keras_model_filename = './h5/appendEnergyTT_wen.h5' #訓練好的模型
tflite_filename = './tflite/appendEnergyTT_wen.tflite' #預建置檔案

# Convert model to TF Lite model
model = models.load_model(keras_model_filename) #載入本來的模型
converter = lite.TFLiteConverter.from_keras_model(model) #將模型載入轉換器
tflite_model = converter.convert() #進行轉換
open(tflite_filename, 'wb').write(tflite_model) #輸出轉換後的模型

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmphg6asl9d\assets


239460

In [44]:
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa
import random

# This gets called every 0.5 seconds
def sound(window,s,m):
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    #print(np.sum(S))
    
    if np.sum(S) >= 0: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=8000, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=True,#True的話，第0個倒頻譜係數被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        val = output_data[0][0]
        
        if val > 0.4:
            data.append(str(m) + "分" + str(s) + "秒，出現:" + str("贅詞") + "，" + str(val))
            #data_yes.append(str(m) + "分" + str(s) + "秒，出現:" + str("贅詞") + "，" + str(val) + ",")
        else:
            data.append(str(m) + "分" + str(s) + "秒，出現:" + str("贅詞") + "，" + str(val))
#main
# Parameters
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/appendEnergyTT_wen.tflite'
#words = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞

data = []
data_yes = []
start = 0 #一開始的索引值
end = 2000 #一開始的索引值
s = 0 #秒
m = 0 #分
duration = 180 #讀音檔的總時間
sample_rate = 8000 #取樣率

#載入音檔
y, sr = librosa.load("./desktop/chen.wav",sr=sample_rate,duration=duration) 

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

while True:
    s = s + 0.25
    if(s >= 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    window[0:2000] = window[2000:4000]
    window[2000:4000] = window[4000:6000]
    window[4000:6000] = window[6000:8000]
    window[6000:8000] = y[start:end]
    
    sound(window,s,m) #呼叫sound()

    if(end == (8000 * duration)): #如果移動到最後，break
        break
    
    start = start + 2000 #向後移動
    end = end + 2000 #向後移動   
    
    
for i in range(len(data)):
        print(data[i])
    
print(len(data))

0分0.25秒，出現:贅詞，0.94904244
0分0.5秒，出現:贅詞，0.0033058482
0分0.75秒，出現:贅詞，0.00076176506
0分1.0秒，出現:贅詞，7.772989e-05
0分1.25秒，出現:贅詞，1.5796659e-05
0分1.5秒，出現:贅詞，3.4986995e-05
0分1.75秒，出現:贅詞，2.0653038e-06
0分2.0秒，出現:贅詞，1.4763333e-09
0分2.25秒，出現:贅詞，3.383291e-12
0分2.5秒，出現:贅詞，3.1504471e-12
0分2.75秒，出現:贅詞，2.0444956e-13
0分3.0秒，出現:贅詞，6.288844e-13
0分3.25秒，出現:贅詞，2.2219565e-10
0分3.5秒，出現:贅詞，2.6428607e-10
0分3.75秒，出現:贅詞，4.2439274e-10
0分4.0秒，出現:贅詞，2.1074407e-09
0分4.25秒，出現:贅詞，5.0160006e-06
0分4.5秒，出現:贅詞，1.3002827e-07
0分4.75秒，出現:贅詞，2.4388925e-07
0分5.0秒，出現:贅詞，7.170511e-08
0分5.25秒，出現:贅詞，3.435016e-07
0分5.5秒，出現:贅詞，7.891805e-09
0分5.75秒，出現:贅詞，1.7052852e-07
0分6.0秒，出現:贅詞，8.688225e-07
0分6.25秒，出現:贅詞，6.0004744e-08
0分6.5秒，出現:贅詞，1.8011409e-10
0分6.75秒，出現:贅詞，6.594373e-11
0分7.0秒，出現:贅詞，1.7258568e-08
0分7.25秒，出現:贅詞，7.3205924e-06
0分7.5秒，出現:贅詞，2.6169488e-05
0分7.75秒，出現:贅詞，2.128834e-13
0分8.0秒，出現:贅詞，7.2636944e-14
0分8.25秒，出現:贅詞，1.0135273e-12
0分8.5秒，出現:贅詞，2.796517e-13
0分8.75秒，出現:贅詞，9.446135e-12
0分9.0秒，出現:贅詞，3.0749941e-10
0分9.25秒，出現:贅詞，2.3661776e-

In [19]:
#slide_window=0.25
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa

# This gets called every 0.5 seconds
def sound(window,s,m):
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    
    if np.sum(S) > 3000: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=8000, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=True,#True的話，第0個倒頻譜係數被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if list_val_max >= 0.8 and list_val_max < 0.9:
   #data_ho.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
             data.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
           # data.append(str(list_val_max))
         
#main
# Parameters
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/appendEnergyTT_wen.tflite'
words = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '他', '你', '吼', '啦', '嗯', '好', '我', '的', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞
data = []
data_ho = []
start = 0 #一開始的索引值
end = 2000 #一開始的索引值
s = 0 #秒
m = 0 #分
duration = 180 #讀音檔的總時間
sample_rate = 8000 #取樣率

#載入音檔
y, sr = librosa.load("./desktop/chen2.wav",sr=sample_rate,duration=duration) 

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


while True:
    s = s + 0.25 #增加秒數
    if(s == 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    #window[:4000] = window[4000:] #把音訊載入window
    #window[4000:] = y[start:end] #把音訊載入window
    window[0:2000] = window[2000:4000]
    window[2000:4000] = window[4000:6000]
    window[4000:6000] = window[6000:8000]
    window[6000:8000] = y[start:end]
    sound(window,s,m) #呼叫sound()

    if(end == (8000 * duration)): #如果移動到最後，break
        break
    
    start = start + 2000 #向後移動
    end = end + 2000 #向後移動    
    
    
for i in range(len(data)):
        print(data[i])
        
print("總共:",len(data))

0分1.0秒，出現:你，預測值:0.8144705891609192
0分3.25秒，出現:我，預測值:0.8558109998703003
0分4.0秒，出現:那，預測值:0.8609471917152405
0分4.5秒，出現:的一個，預測值:0.8529371619224548
0分5.25秒，出現:我，預測值:0.8732877969741821
0分10.75秒，出現:我，預測值:0.8623384237289429
0分12.25秒，出現:我，預測值:0.8259959816932678
0分13.25秒，出現:你，預測值:0.8380088806152344
0分14.75秒，出現:的一個，預測值:0.8247097730636597
0分15.0秒，出現:的這個，預測值:0.8152815699577332
0分17.5秒，出現:我，預測值:0.8642467260360718
0分19.5秒，出現:你，預測值:0.8537974953651428
0分22.25秒，出現:ㄟ，預測值:0.8161870241165161
0分22.75秒，出現:你，預測值:0.8976947069168091
0分29.0秒，出現:我，預測值:0.8791391849517822
0分30.5秒，出現:的一個，預測值:0.8202406167984009
0分30.75秒，出現:你，預測值:0.8615965843200684
0分32.75秒，出現:我，預測值:0.813188374042511
0分33.0秒，出現:我，預測值:0.8136052489280701
0分33.5秒，出現:我，預測值:0.8641261458396912
0分36.25秒，出現:你，預測值:0.8368300795555115
0分37.5秒，出現:的這個，預測值:0.8052206635475159
0分37.75秒，出現:你，預測值:0.8632212281227112
0分42.75秒，出現:我，預測值:0.8972828984260559
0分43.0秒，出現:我，預測值:0.840818464756012
0分49.5秒，出現:ㄟ，預測值:0.8279343843460083
0分49.75秒，出現:我，預測值:0.8911739587783813
0分54.5秒，出現: